<a href="https://colab.research.google.com/github/mbalbi/ciencia_de_datos/blob/main/notebooks/practica_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Clase 3: Adquisición y procesamiento de datos

##EDA: Análisis exploratorio de datos

El análisis exploratorio de datos, EDA, es una etapa crítica en la ciencia de datos y es la base de cualquier uso posterior que se le dé a los mismos.

Permite:
- Comprender el problema y la calidad de la información
- Limpiar los datos
- Comprobar supuestos

**Haremos uso de una nueva librería: pandas.** 
Esta permite leer facilmente archivos en distintos formatos y en bases de datos SQL, se basa en los arrays de NumPy y permite acceder a los datos mediante un índice, o nombre de filas y columnas.

Tipos de datos de Pandas:
- Series: una dimensión
- DataFrame: dos dimensiones (tablas)
- Panel: tres dimensiones (cubos)

### Carga de los datos: Dataset siniestros

Se desea tomar acciones para reducir los accidentes de tránsito con heridos graves y mortales que ocurren en la ciudad de Buenos Aires, por lo que se analizarán los registros (año 2015-2018) que se encuentran en el siguiente link: 
https://data.buenosaires.gob.ar/dataset/victimas-siniestros-viales

In [ ]:
#Se importan las librerías:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Abrimos un archivo local, es decir, que ha sido descargado en su computadora
from google.colab import files
uploaded = files.upload()

In [ ]:
import io

#Se crea un DataFrame
df_siniestros = pd.read_csv(io.BytesIO(uploaded['Victimas_siniestros_2015-2018.csv']))

df_siniestros.head()

In [ ]:
# Veremos el tamaño de la data y el nombre de la columnas
print('Cantidad de Filas y columnas:', df_siniestros.shape)
print('Nombre columnas:', df_siniestros.columns)

In [ ]:
# Podemos ver, por columna, cuantos valores no nulos hay y el tipo de dato que contiene
df_siniestros.info()

In [ ]:
#Modificar el tipo de dato:
df_siniestros['fecha'] = pd.to_datetime(df_siniestros['fecha']) # astype() es otra forma
df_siniestros.dtypes

In [ ]:
df_siniestros['sexo'].unique()

##### **NaN = 'Not a Number'**: Indica un valor faltante o nulo de tipo float

In [ ]:
a = np.nan
print(type(a))

Si queremos convertir estos valores, existen varios métodos útiles para detectar, eliminar y reemplazar valores nulos en las estructuras de datos de Pandas. Están:

- `isnull()`: genera un booleano (`True` - `False`) indicando los valores faltantes
- `notnull()`: lo opuesto a `isnull()`
- `dropna()`: descarta los valores faltantes.
- `fillna()`: devuelve una copia de los datos con los valores faltantes reemplazados por un valor válido. 

En una breve exploración, mostraremos estos métodos:

In [ ]:
data = pd.Series([15, np.nan, 'hello'])

data.isnull()

In [ ]:
data.notnull()

In [ ]:
data_without_NaN = data.dropna()
data_without_NaN

In [ ]:
data_replace_NaN = data.fillna(0)
data_replace_NaN

##### Descripción estadística y duplicados

In [ ]:
# Descripción estadística de los datos numéricos: cantidad, media, desvío estándar, percentiles, valores máximo y mínimo.
df_siniestros.describe()

# ¡OJO! ¿Qué significa count?

In [ ]:
#Encontrar filas duplicadas
df_siniestros.duplicated().sum()

In [ ]:
#Elimino las filas duplicadas
df_siniestros.drop_duplicates()

In [ ]:
#Analicemos con mayor profundidad, qué ocurre en las filas 33230 y 33232:
#pd.options.display.max_columns = 29 #para ver todas las columnas 
df_siniestros.iloc[33230:33232]

In [ ]:
#Podemos especificar qué columnas no deben repetirse, antes de droppear
df_unique = df_siniestros.drop_duplicates(
    subset=['causa', 'mes', 'periodo', 'fecha', 'hora', 'lugar_hecho', 'direccion_normalizada', 'tipo_calle',
            'direccion_normalizada_arcgis', 'calle1', 'altura', 'calle2', 'codigo_calle', 'codigo_cruce', 'geocodificacion', 
            'semestre', 'x', 'y', 'geom', 'cantidad_victimas', 'comuna', 'geom_3857', 'tipo_colision1', 
            'participantes_victimas', 'participantes_acusados'])

df_unique.shape

## Análisis de variables:
Los datos pueden ser de dos tipos:

- Cuantitativos: se representan por números discretos (cantidad de personas) o continuos (coordenadas geográficas).

- Categóricos: datos cualitativos que pueden ser ordinales (bajo, medio, alto) o no ordinal (sexo, estado civil, ciudad natal)

### Correlación

In [ ]:
# Matriz de correlación
df_siniestros.corr()

In [ ]:
# Hacer un heatmap fácil de leer con matplotlib, requiere de un código largo que puede evitarse con otras librerías.
plt.imshow(df_siniestros.corr(), cmap='BuPu')
plt.title('Correlación')
plt.colorbar()  
plt.show()

In [ ]:
# Gráfico de calor para la matriz de correlación
sns.heatmap(df_siniestros.corr(), cmap='BuPu', fmt='.2f', annot=True, linewidths=.6)

### Auto-correlación

Los gráficos de autocorrelación permiten verificar la aleatoriedad en un conjunto de datos, midiendo si se correlacionan los valores actuales contra los pasados:

$$\rho _{XX}(t_{1},t_{2}) = \tfrac{E\left[ ( X_{t1}-\mu_{t1} )\cdot \left( X_{t2}-\mu _{t2}\right) \right]}{\sigma_{t1}\sigma_{t2}}
$$

Es decir, se calcula la correlación entre un valor y la versión desplazada en el tiempo. Ese desplazamiento temporal, se conoce como desfase o lag.


In [ ]:
altura = df_siniestros['altura'].dropna()

# Gráfito de Autocorrelación
plt.acorr(altura, normed=True, maxlags=10)

### **Histogramas** de variables numéricas

In [ ]:
# Histograma con Seaborn
sns.histplot(data = df_siniestros, x='edad', bins='sturges', stat='probability').set(title='Histograma de edad')

In [ ]:
# Histograma con Seaborn
sns.histplot(data=df_siniestros, x='edad', hue='sexo', bins='sturges', multiple='stack', stat='probability').set(title='Histograma de edad por sexo')

In [ ]:
# Histograma suavizado (densidad) con Seaborn
sns.kdeplot(data=df_siniestros, x='edad', hue='sexo', fill=True).set(title='Histograma de edad por sexo')

### Pair plots

Genera una matriz de gráficos donde la diagonal está compuesta por histogramas y el resto muestra la relación entre pares de variables

In [ ]:
vars = ['edad', 'altura', 'codigo_calle', 'cantidad_victimas']

sns.pairplot(df_siniestros, x_vars=vars, y_vars=vars, hue='sexo')

### **Boxplots**

Un boxplot muestra la distribución de datos cuantitativos de una manera que facilita las comparaciones entre variables o entre niveles de una variable categórica. Está constituído por:

  * **La caja**: es un rectángulo que abarca el rango intercuartílico (RIC) de la distribución, es decir, el tramo de la escala que va desde el primer cuartil (C1: 25%) al tercer cuartil (C3=75%), abarcando el 50% de las observaciones centrales.

  * **La Mediana**: Se dibuja mediante una línea dentro de la caja.

  * **Los Bigotes**: Son líneas que salen a los costados de la caja hasta el valor mínimo o máximo, según corresponda.
  
  * **Los valores atípicos o outliers**: punto que se encuentran más allá del mínimo/máximo, por l que no se lo incluye como parte de la distribución.

  

Seaborn para calcular los outliers, utiliza el método llamado **múltiplo del rango intercuartílico**:

$$ Mínimo = C1 – 1.5 * RIC $$

$$ Máximo = C3 + 1.5 * RIC $$

modificando el parámetro `whis = 1.5` podemos cambiar el valor por default

In [ ]:
# Boxplots con Seaborn
sns.boxplot(data=df_siniestros, x='edad').set(title='Bloxplot de edad')

In [ ]:
sns.boxplot(data=df_siniestros, x='edad', y='sexo').set(title='Bloxplot de edad por sexo')

In [ ]:
#Veamos los valores únicos para las edades:
df_siniestros['edad'].unique()

In [ ]:
# Filtramos las filas para edad > 99
df_siniestros[df_siniestros['edad'] > 99]

### **Histogramas** de variables categóricas

In [ ]:
df_siniestros['causa'].value_counts()

In [ ]:
# Histograma con Seaborn
sns.histplot(data = df_siniestros, x='causa', bins='sturges', stat='count').set(title='Histograma de causas')

In [ ]:
# Gráfico de barras con Seaborn
sns.countplot(data = df_siniestros, x='causa').set(title='Número de accidente por Causas')

## Limpieza:

* Observe la columna **tipo** de valores categóricos. ¿Es necesaria alguna corrección en los datos?

In [ ]:
#Contabilizar la frecuenta por valor único
df_siniestros['tipo'].value_counts()

In [ ]:
# Reemplazaremos ciertos valores
df_siniestros['tipo'] = df_siniestros['tipo'].replace('tren / subte', 'tren / subte / tranvia')
df_siniestros['tipo'] = df_siniestros['tipo'].replace('auto pfa / movil / gendarmeria / metropolitana / moto movil', 'fuerza seguridad')

In [ ]:
df_siniestros['tipo'].value_counts()

* Analizar, por **sexo** y **rol**, las personas que participaron en los accidentes

In [ ]:
df_siniestros['rol'].unique()

In [ ]:
# Tabla de contingencia
pd.crosstab(index=df_siniestros['sexo'], columns=df_siniestros['rol'], margins=True)

* Crear un DataFrame que analice, por **sexo**, **rol**, **tipo_calle**, las personas que participaron en los accidentes.

In [ ]:
# Con la función de groupby generamos un nuevo DataFrame
df_causa = df_siniestros.groupby(['sexo', 'tipo_calle', 'rol']).agg('size')
df_causa = pd.DataFrame(df_causa).rename(columns={0:'number'})
df_causa

* Realizar un análisis temporal por **año** y **semestre**:

In [ ]:
sns.countplot(data=df_siniestros, x='periodo', hue='semestre').set(title='Número de accidentados por año y semestre')

* Realizar un análisis temporal por **año** y **semestre**, dividido según el **tipo de calle**

In [ ]:
sns.catplot(data=df_siniestros, x='periodo', hue='semestre', col='tipo_calle', kind='count')

* Realizar un gráfico de línea que muestre el porcentaje de accidentes a lo largo de cada año. 
Crear un DataFrame que calcule el porcentaje de los accidentados, siendo la filas: **año** y las columnas:  **mes**.

In [ ]:
df_periodo = df_siniestros.groupby(['periodo', 'mes']).agg('size')
#df_periodo = pd.DataFrame(df_periodo).rename(columns={0:'number'})
#df_periodo['percent'] = df_periodo.groupby(level=0).apply(lambda x: 100*(x/x.sum()))
#df_periodo = df_periodo.reset_index()
df_periodo

In [ ]:
#Grafico los accidentados a lo largo del año
sns.lineplot(data=df_periodo, x='mes', y='percent', hue='periodo').set(title='Porcentaje de accidentados a lo largo del año')

In [ ]:
#Filtramos el dataframe:
df_periodo = df_periodo[df_periodo['periodo'] != 2018]

sns.lineplot(data=df_periodo, x='mes', y='percent', hue='periodo').set(title='Porcentaje de accidentados a lo largo del año')

In [ ]:
#Reorganizo el DataFrame como fue solicitado
df_periodo = df_periodo.pivot(index='periodo', columns='mes', values='number')

  *   Agregar la columna **grupo_etario** en el DataFrame, que agrupe los accidentados según su edad en:
*   Menor: menores de 17 años
*   Joven adulto: entre 17 y 35 años inclusive
*   Adulto: entre 35 y 65 años inclusive
*   Adulto mayor: mayores de 65 años

In [ ]:
df_siniestros['edad'].unique()

In [ ]:
# crear lista de las condiciones
conditions = [
    (df_siniestros['edad'] <= 17),
    (df_siniestros['edad'] > 17) & (df_siniestros['edad'] <= 35),
    (df_siniestros['edad'] > 35) & (df_siniestros['edad'] <= 65),
    (df_siniestros['edad'] > 65)
    ]

# crear lista de los valores que se quieren asignar a cada condición
values = ['menor', 'joven_adulto', 'adulto', 'adulto_mayor']

# crear columna nueva y usar np.select() para asignarle valores usando las listas como argumentos
df_siniestros['grupo_etario'] = np.select(conditions, values)

# mostrar el DataFrame actualizado
df_siniestros

In [ ]:
#No tenemos la edad de todos, entonces si aparece NaN, quedó 0:
df_siniestros['grupo_etario'].unique()

  *   Agregar la columna **barrios** según la comuna donde ocurrió el accidente:

  Para ello, se debió descargar la información. https://es.wikipedia.org/wiki/Comunas_de_la_ciudad_de_Buenos_Aires

In [ ]:
#Abrimos un archivo local, es decir, que ha sido descargado en su computadora
from google.colab import files
uploaded = files.upload()

In [ ]:
#Se crea un DataFrame
list_comuna = pd.read_html(io.BytesIO(uploaded["Comunas de la ciudad de Buenos Aires - Wikipedia, la enciclopedia libre.html"]))
df_comuna = list_comuna[0]

df_comuna

In [ ]:
df_comuna.columns

In [ ]:
#Me quedo sólo con algunas columnas del DataFrame
df_comuna = df_comuna[['Comuna', 'Población (2010) [4]​', 'Superficie (km²)[5]​', 'Barrios[2]​']]

#Renombro las columnas
df_comuna = df_comuna.rename(columns={'Comuna': 'comuna',
                                'Población (2010) [4]​': 'Población', 
                                'Superficie (km²)[5]​': 'Superficie', 
                                'Barrios[2]​': 'Barrios'})

#Sólo me quedo con el número de la comuna en la columna comuna.
df_comuna['comuna'] = df_comuna['comuna'].str.split().str[-1].astype(int)
df_comuna

Para unir los DataFrames, utilizo .merge(), cuyo parámetro how puede ser:



In [ ]:
from matplotlib_venn import venn2, venn2_circles
from matplotlib import pyplot as plt

figure, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(10,10))

# Left Join
v1 = venn2(subsets=(3, 3, 1), ax=ax1)
c1 = venn2_circles(subsets=(3, 3, 1), ax=ax1)

for area in ['01', '10', '11']:
    color = 'skyblue' if area != '01' else 'white'
    v1.get_patch_by_id(area).set_color(color)
    v1.get_patch_by_id(area).set_alpha(1)
    txt = v1.get_label_by_id(area)
    if txt: txt.set_text('')

ax1.set_facecolor('white')
ax1.set_title('Left Join')

# Right Join
v2 = venn2(subsets=(3, 3, 1), ax=ax2)
c2 = venn2_circles(subsets=(3, 3, 1), ax=ax2)

for area in ['01', '10', '11']:
    color = 'skyblue' if area != '10' else 'white'
    v2.get_patch_by_id(area).set_color(color)
    v2.get_patch_by_id(area).set_alpha(1)
    txt = v2.get_label_by_id(area)
    if txt: txt.set_text('')

ax2.set_facecolor('white')
ax2.set_title('Right Join')

# Inner Join
v3 = venn2(subsets=(3, 3, 1), ax=ax3)
c3 = venn2_circles(subsets=(3, 3, 1), ax=ax3)

for area in ['01', '10', '11']:
    color = 'skyblue' if area == '11' else 'white'
    v3.get_patch_by_id(area).set_color(color)
    v3.get_patch_by_id(area).set_alpha(1)
    txt = v3.get_label_by_id(area)
    if txt: txt.set_text('')

ax3.set_facecolor('white')
ax3.set_title("Inner Join")

# Inner Join
v4 = venn2(subsets=(3, 3, 1), ax=ax4)
c4 = venn2_circles(subsets=(3, 3, 1), ax=ax4)

for area in ['01', '10', '11']:
    v4.get_patch_by_id(area).set_color('skyblue')
    v4.get_patch_by_id(area).set_alpha(1)
    txt = v4.get_label_by_id(area)
    if txt: txt.set_text('')

ax4.set_facecolor('white')
ax4.set_title("Outer Join")

plt.show()

In [ ]:
#Uno los dos Dataframes
df_siniestros_nuevo = df_siniestros.merge(df_comuna, how='left', on='comuna')
df_siniestros_nuevo

In [ ]:
#Filtro por los barrios que me interesan
df_siniestros_nuevo.where(df_siniestros_nuevo['Barrios'].str.contains('Boca', regex=False)).dropna(how='all')